In [1]:
NODES = 1
RANK = 0
EPOCHS = 10
BATCH = 64

In [2]:
import tensorflow as  tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow import keras
%load_ext tensorboard

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

# print(x_train[0].shape)

def build_rnn():
    model = Sequential()
    
    model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax')) # activation must be softmax for categorical cross entropy loss
    
    opt = tf.keras.optimizers.Adam(lr=1e-3, decay=0)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,
             metrics=['accuracy'])
  
    return model

In [4]:
if NODES > 1:
    workers = []
    if NODES == 2:
        workers = ["10.0.1.121:8890", "10.0.1.104:8890"]
    else:
        workers = ["10.0.1.121:8890", "10.0.1.104:8890", "10.0.1.46:8890"]
    import json
    import os
    os.environ['TF_CONFIG'] = json.dumps({
        'cluster': {
            'worker': workers
        },
        'task': {'type': 'worker', 'index': RANK}
    })
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    with strategy.scope():
        net = build_cnn()
else:
    net = build_rnn()

In [5]:
logdir = 'logs/tf_RNN_{}batch_node{}of{}_4CPUs'.format(BATCH,RANK+1,NODES)
!rm -rf {logdir}
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True,update_freq='batch',
                                                  profile_batch=(1,60000//BATCH))

In [6]:
net.fit(x_train, y_train, batch_size = BATCH,
            epochs=EPOCHS, validation_data=(x_test, y_test),
            callbacks=[tensorboard_callback])

Epoch 1/10
936/938 [============================>.] - ETA: 0s - accuracy: 0.7524 - loss: 0.7578WARNING:tensorflow:From /home/ubuntu/night/lib/python3.5/site-packages/tensorflow/python/ops/summary_ops_v2.py:1271: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
938/938 [==============================] - 52s 56ms/step - accuracy: 0.7526 - loss: 0.7569 - val_accuracy: 0.9454 - val_loss: 0.1848
Epoch 2/10
938/938 [==============================] - 24s 25ms/step - accuracy: 0.9455 - loss: 0.1993 - val_accuracy: 0.9625 - val_loss: 0.1352
Epoch 3/10
938/938 [==============================] - 25s 26ms/step - accuracy: 0.9638 - loss: 0.1292 - val_accuracy: 0.9673 - val_loss: 0.1116
Epoch 4/10
938/938 [==============================] - 26s 28ms/step - accuracy: 0.9714 - loss: 0.1055 - val_accuracy: 0.9787 - val_loss: 0.0787
Epoch 5/10
938/938 [==============================] - 

In [7]:
# %tensorboard --logdir logs --port 6006 --bind_all

In [8]:
# model.save('Minst_Lstm.model')